In [190]:
using LowLevelFEM, SparseArrays
gmsh.initialize()

gmsh.open("coord-test.geo")

Info    : Reading 'coord-test.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 40%] Meshing curve 2 (Line)
Info    : [ 70%] Meshing curve 3 (Line)
Info    : Done meshing 1D (Wall 0.00033869s, CPU 0.000317s)
Info    : 3 nodes 6 elements
Info    : Done reading 'coord-test.geo'


In [191]:
mat = material("truss")
prob = Problem([mat], type=:Truss)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 2)


Problem("coord-test", :Truss, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("truss", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 3)

In [192]:
cs0 = CoordinateSystem([-√3, 1, 0], [-1, 1, 0])
Q = rotateNodes(prob, "left", cs0)
Q *= rotateNodes(prob, "top", cs0)

LowLevelFEM.Transformation(sparse([1, 2, 1, 2, 3, 4, 5, 6, 7, 8, 7, 8, 9], [1, 1, 2, 2, 3, 4, 5, 6, 7, 7, 8, 8, 9], [-0.8660254037844387, 0.5000000000000001, 0.5000000000000009, 0.8660254037844382, -1.0000000000000002, 1.0, 1.0, 1.0, -0.8660254037844387, 0.5000000000000001, 0.5000000000000009, 0.8660254037844382, -1.0000000000000002], 9, 9), 3, 3)

In [193]:
Q.T

9×9 SparseMatrixCSC{Float64, Int64} with 13 stored entries:
 -0.866025  0.5         ⋅    ⋅    ⋅    ⋅     ⋅         ⋅          ⋅ 
  0.5       0.866025    ⋅    ⋅    ⋅    ⋅     ⋅         ⋅          ⋅ 
   ⋅         ⋅        -1.0   ⋅    ⋅    ⋅     ⋅         ⋅          ⋅ 
   ⋅         ⋅          ⋅   1.0   ⋅    ⋅     ⋅         ⋅          ⋅ 
   ⋅         ⋅          ⋅    ⋅   1.0   ⋅     ⋅         ⋅          ⋅ 
   ⋅         ⋅          ⋅    ⋅    ⋅   1.0    ⋅         ⋅          ⋅ 
   ⋅         ⋅          ⋅    ⋅    ⋅    ⋅   -0.866025  0.5         ⋅ 
   ⋅         ⋅          ⋅    ⋅    ⋅    ⋅    0.5       0.866025    ⋅ 
   ⋅         ⋅          ⋅    ⋅    ⋅    ⋅     ⋅         ⋅        -1.0

In [194]:
supp1 = displacementConstraint("left", uy=0)
supp2 = displacementConstraint("right", ux=0, uy=0)
supp0 = displacementConstraint("truss", uz=0)

load1 = load("top", fy=-1000)

("top", 0, -1000, 0)

In [195]:
K = stiffnessMatrix(prob)

9×9 SparseMatrixCSC{Float64, Int64} with 22 stored entries:
  200000.0         ⋅    ⋅   …   ⋅         ⋅          ⋅          ⋅ 
        ⋅    200000.0   ⋅       ⋅         ⋅   -200000.0         ⋅ 
        ⋅          ⋅    ⋅       ⋅         ⋅          ⋅          ⋅ 
 -200000.0         ⋅    ⋅       ⋅   -70710.7    70710.7         ⋅ 
        ⋅          ⋅    ⋅       ⋅    70710.7   -70710.7         ⋅ 
        ⋅          ⋅    ⋅   …   ⋅         ⋅          ⋅          ⋅ 
        ⋅          ⋅    ⋅       ⋅    70710.7   -70710.7         ⋅ 
        ⋅   -200000.0   ⋅       ⋅   -70710.7        2.70711e5   ⋅ 
        ⋅          ⋅    ⋅       ⋅         ⋅          ⋅          ⋅ 

In [196]:
f = loadVector(prob, [load1])

9×1 Matrix{Float64}:
     0.0
     0.0
     0.0
     0.0
     0.0
     0.0
     0.0
 -1000.0
     0.0

In [197]:
Q' * f

9×1 Matrix{Float64}:
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
 -500.0000000000001
 -866.0254037844381
    0.0

In [198]:
K1, f1 = applyBoundaryConditions(Q' * K * Q, Q' * f, [supp0, supp1, supp2])

9×9 SparseMatrixCSC{Float64, Int64} with 15 stored entries:
 200000.0   ⋅    ⋅    ⋅    ⋅    ⋅   -50000.0        -86602.5         ⋅ 
       ⋅   1.0   ⋅    ⋅    ⋅    ⋅         ⋅               ⋅          ⋅ 
       ⋅    ⋅   1.0   ⋅    ⋅    ⋅         ⋅               ⋅          ⋅ 
       ⋅    ⋅    ⋅   1.0   ⋅    ⋅         ⋅               ⋅          ⋅ 
       ⋅    ⋅    ⋅    ⋅   1.0   ⋅         ⋅               ⋅          ⋅ 
       ⋅    ⋅    ⋅    ⋅    ⋅   1.0        ⋅               ⋅          ⋅ 
 -50000.0   ⋅    ⋅    ⋅    ⋅    ⋅        1.81948e5       1.21958e5   ⋅ 
 -86602.5   ⋅    ⋅    ⋅    ⋅    ⋅        1.21958e5       1.59473e5   ⋅ 
       ⋅    ⋅    ⋅    ⋅    ⋅    ⋅         ⋅               ⋅         1.0

9×1 Matrix{Float64}:
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
 -500.0000000000001
 -866.0254037844381
    0.0

(, )

In [199]:
q = (Q * K1 * Q') \ (Q * f1)

9×1 Matrix{Float64}:
  0.002886751345996453
 -0.001666666666582964
  0.0
  0.0
  0.0
  0.0
 -0.006666666666666652
 -0.006666666666666667
  0.0

In [200]:
showDoFResults(q)

0

In [201]:
f2 = K * q

9×1 Matrix{Float64}:
   577.3502691992907
  1000.0000000167406
     0.0
  -577.3502691992917
     1.0800249583553523e-12
     0.0
     1.0800249583553523e-12
 -1000.0000000167419
     0.0

In [202]:
showDoFResults(f2)

1

In [203]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [204]:
gmsh.finalize()